# What is a for loop?

These two code blocks do the same

In [25]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [86]:
range(10)

range(0, 10)

In [26]:
range_iter = iter(range(10))
while True:
    try:
        i = next(range_iter)
    except StopIteration:
        break
    
    print(i)

0
1
2
3
4
5
6
7
8
9


How do we create an iterable object then?

## Start with the Polygon class

In [2]:
class Polygon:
    def __init__(self, points):
        """Create a polygon from the list of numbers"""
        self.points = [[float(x), float(y)] for x, y in points]

    def __repr__(self):
        return f"Polygon with {len(self)} points at {id(self)}"

    def __len__(self):
        return len(self.points)

    def __getitem__(self, item):
        return self.points[item]

    def __setitem__(self, item, value):
        x, y = value
        self.points[item] = [float(x), float(y)]

    def __add__(self, value):
        x_add, y_add = value
        points = [[x + float(x_add), y + float(y_add)] for x, y in self.points]
        return Polygon (points)

    def __radd__(self, value):
        return self + value

    def draw(self):
        raise NotImplementedError
    

## Make Polygon into an iterator

In [9]:
class Polygon:
    def __init__(self, points):
        """Create a polygon from the list of numbers"""
        self.points = [[float(x), float(y)] for x, y in points]
        self.current_idx = None

    def __repr__(self):
        return f"Polygon with {len(self)} points at {id(self)}"

    def __len__(self):
        return len(self.points)

    def __getitem__(self, item):
        return self.points[item]

    def __setitem__(self, item, value):
        x, y = value
        self.points[item] = [float(x), float(y)]

    def __add__(self, value):
        x_add, y_add = value
        points = [[x + float(x_add), y + float(y_add)] for x, y in self.points]
        return Polygon (points)

    def __radd__(self, value):
        return self + value

    def draw(self):
        raise NotImplementedError
    
    # New part
    def __iter__(self):
        self.current_idx = 0
        return self
    
    def __next__(self):        
        if self.current_idx == len(self):
            raise StopIteration

        first_point = self[self.current_idx]
        second_point = self[(self.current_idx + 1) % len(self)]
        self.current_idx += 1
        return [first_point, second_point]


In [11]:
for line in Polygon([[3, 2], [4, 5], [7, 6]]):
    print(i)

[[7.0, 6.0], [3.0, 2.0]]
[[7.0, 6.0], [3.0, 2.0]]
[[7.0, 6.0], [3.0, 2.0]]


## Add Iterator class
This is not ideal, we don't want the iterator and the object to be the same.

In [31]:
class PolygonLineIterator:
    def __init__(self, polygon):
        self.polygon = polygon
        self.idx = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.idx == len(self.polygon):
            raise StopIteration

        first_point = self.polygon[self.idx]
        second_point = self.polygon[(self.idx + 1) % len(self.polygon)]
        self.idx += 1
        return [first_point, second_point]

class Polygon:
    def __init__(self, points):
        """Create a polygon from the list of numbers"""
        self.points = [[float(x), float(y)] for x, y in points]
        self.current_idx = None

    def __repr__(self):
        return f"Polygon with {len(self)} points at {id(self)}"

    def __len__(self):
        return len(self.points)

    def __getitem__(self, item):
        return self.points[item]

    def __setitem__(self, item, value):
        x, y = value
        self.points[item] = [float(x), float(y)]

    def __add__(self, value):
        x_add, y_add = value
        points = [[x + float(x_add), y + float(y_add)] for x, y in self.points]
        return Polygon(points)

    def __radd__(self, value):
        return self + value

    def draw(self):
        raise NotImplementedError
    
    def __iter__(self):
        return iter(PolygonLineIterator(self))

In [32]:
for line in Polygon([[3, 2], [4, 5], [7, 6]]):
    print(line)

[[3.0, 2.0], [4.0, 5.0]]
[[4.0, 5.0], [7.0, 6.0]]
[[7.0, 6.0], [3.0, 2.0]]


## Add point iterator

In [33]:
class PolygonPointIterator:
    def __init__(self, polygon):
        self.polygon = polygon
        self.idx = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.idx == len(self.polygon):
            raise StopIteration
        point = self.polygon[self.idx]
        self.idx += 1
        return point
    
class PolygonLineIterator:
    def __init__(self, polygon):
        self.polygon = polygon
        self.idx = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.idx == len(self.polygon):
            raise StopIteration

        first_point = self.polygon[self.idx]
        second_point = self.polygon[(self.idx + 1) % len(self.polygon)]
        self.idx += 1
        return [first_point, second_point]

class Polygon:
    def __init__(self, points):
        """Create a polygon from the list of numbers"""
        self.points = [[float(x), float(y)] for x, y in points]
        self.current_idx = None

    def __repr__(self):
        return f"Polygon with {len(self)} points at {id(self)}"

    def __len__(self):
        return len(self.points)

    def __getitem__(self, item):
        return self.points[item]

    def __setitem__(self, item, value):
        x, y = value
        self.points[item] = [float(x), float(y)]

    def __add__(self, value):
        x_add, y_add = value
        points = [[x + float(x_add), y + float(y_add)] for x, y in self.points]
        return Polygon(points)

    def __radd__(self, value):
        return self + value

    def draw(self):
        raise NotImplementedError
    
    def __iter__(self):
        return PolygonPointIterator(self)
    
    def iterlines(self):
        return PolygonLineIterator(self)

In [34]:
for point in Polygon([[1, 2], [3, 4], [5, 6]]):
    print(point)

[1.0, 2.0]
[3.0, 4.0]
[5.0, 6.0]


In [35]:
for line in Polygon([[1, 2], [3, 4], [5, 6]]).iterlines():
    print(line)

[[1.0, 2.0], [3.0, 4.0]]
[[3.0, 4.0], [5.0, 6.0]]
[[5.0, 6.0], [1.0, 2.0]]


# Generators and lazy evaluation
Generators are another way to create iterators.

In [39]:
def even_numbers(n):
    """Returns a list with all even numbers less than n.
    """
    return [number for number in range(n) if number % 2 == 0]


for i in even_numbers(10):
    print(i)

0
2
4
6
8


**Problem with this approach:**
  * Compute all numbers at once, takes long time in the beginning
  * Stores all elements in RAM
 
**Benefits of this approach:**
  * Lists are easy to work with
  * Can access any number at any time
  
Alternative: **Generators**

In [42]:
def even_numbers_gen(n):
    """Yields each even number less than n.
    """
    for number in range(n):
        if number % 2 == 0:
            yield number

for i in even_numbers_gen(10):
    print(i)

0
2
4
6
8


#### What happens here?

A Generator is a way to create iterables in Python. The code above runs as follows.
 * First we create a generator, `even_numbers_gen`.
 * Then, we iterate over it. The first iteration, the code runs til the first `yield` statement.
   * Now, number is equal to 0
 * Then, the yield statement works as if it were a return, and i is set to whatever the generator yields.
   * Now i is equal to 0
 * The code inside the for loop is ran, and once a new iteration starts, the code inside the generator
   is run *starting from the yield statment*.
   * number starts equal to 0, the inner loop is ran for two iterations before the next yield statment is reached.
     number is now equal to 2
 * After the generator reaches its next yield statement, i is set equal to whatever the generator yielded.
   * now i is equal to 2


**Benefits of this approach:**
  * Only one element in RAM at once
  * It is clear that the sequence is meant to be iterated over
  
**Downsides of this approach:**
  * Each iteration takes longer than if the list were precomputed
  * Can easily be integrated in classes
  
### A more common pattern for generators

In [44]:
def odd_numbers(n):
    """Yields each odd number less than n.
    """
    number = 0
    while True:
        number += 1
        if number >= n:
            return
        elif number % 2 == 1:
            yield number

for i in odd_numbers(10):
    print(i)

1
3
5
7
9


This pattern lets us iterate over sequences without knowing how long they are.

The idea of generators, namely only computing the numbers when needed is called lazy evaluation.


### Let us create a useful generator.

In [45]:
%%writefile file.txt
hei
hallo
heisann

Writing file.txt


In [55]:
with open('file.txt') as f:
    for i in range(10):
        print(repr(f.readline()))    

'hei\n'
'hallo\n'
'heisann\n'
''
''
''
''
''
''
''


In [57]:
def iter_lines(file_object):
    while True:
        line = file_object.readline()
        if line == '':
            return
        yield line

In [58]:
with open('file.txt') as f:
    for line in iter_lines(f):
        print(line)

hei

hallo

heisann



Why is this useful, why can we not simply use `f.readlines()` instead?
 * `f.readlines()` returns a list of lines, which won't work for really long files.

## Generator comprehensions

We can also have generator comprehensions, akin to list comprehensions

In [62]:
def integers():
    n = 0
    while True:
        yield n
        n += 1

odd_integers = (i for i in integers() if i % 2 == 1)

for integer in odd_integers:
    if integer > 10:
        break
    print(integer)

1
3
5
7
9


In [69]:
def is_prime(n):
    for i in range(2, n):
        if n % i == 0:
            return False
    return True


primes = (i for i in integers() if is_prime(i))

for prime in primes:
    if prime > 50:
        break
    print(prime)

0
1
2
3
5
7
11
13
17
19
23
29
31
37
41
43
47


## Generators in classes

We can use generators instead of the iterator classes we used for the Polygon class

In [72]:
class PolygonPointIterator:
    def __init__(self, polygon):
        self.polygon = polygon
        self.idx = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.idx == len(self.polygon):
            raise StopIteration
        point = self.polygon[self.idx]
        self.idx += 1
        return point


class Polygon:
    def __init__(self, points):
        """Create a polygon from the list of numbers"""
        self.points = [[float(x), float(y)] for x, y in points]
        self.current_idx = None

    def __repr__(self):
        return f"Polygon with {len(self)} points at {id(self)}"

    def __len__(self):
        return len(self.points)

    def __getitem__(self, item):
        return self.points[item]

    def __setitem__(self, item, value):
        x, y = value
        self.points[item] = [float(x), float(y)]

    def __add__(self, value):
        x_add, y_add = value
        points = [[x + float(x_add), y + float(y_add)] for x, y in self.points]
        return Polygon(points)

    def __radd__(self, value):
        return self + value

    def draw(self):
        raise NotImplementedError
    
    def __iter__(self):
        return PolygonPointIterator(self)
    
    def iterlines(self):
        for idx, point in enumerate(self):
            next_point = self[(idx + 1) % len(self)]
            yield [point, next_point]

In [73]:
for line in Polygon([[3, 4], [7, 8], [3, 1]]).iterlines():
    print(line)

[[3.0, 4.0], [7.0, 8.0]]
[[7.0, 8.0], [3.0, 1.0]]
[[3.0, 1.0], [3.0, 4.0]]


## Higher order functions

Functions in Python are just like variables. And in the same way that we can modify variables, we can modify functions to. Let us look at an example

In [77]:
def add(x, y):
    return x + y

def multiply(x, y):
    return x*y

In [78]:
def add_3_to_result(f, x, y):
    return f(x, y) + 3


In [80]:
add_3_to_result(add, 2, 2)

7

In [81]:
add_3_to_result(multiply, 4, 4)

19

Here we have seen our first example of a higher-order function. In Python, a higher order
function is a function that either takes a function as input, returns a function, or both.

Let us now look at the latter case.

In [82]:
def add_n(n):
    def add(x):
        return x + n
    return add

add_3 = add_n(3)
print(add_3(4))

7


What happens here? Let us talk about closures.

A function has access to the variable in the closure in which it was created. Normally, the
closure is the global frame. That is, a function has access to all global variables.

When we define a higher-order function, however, the closure is not the global frame, but the
frame in which the function was created. Thus, a function has access to all local variables 
inside the function where it was created, as well as all global variables.

## Currying

This idea let's us use the concept of currying to split the inputs to a function.

In [83]:
def multiply(x, y):
    return x*y

def curried_multiply(x):
    def local_multiply(y):
        return x*y
    return local_multiply

print(multiply(3, 2))
print(curried_multiply(3)(2))

6
6


Here we see the concept of currying in action. This concept can be very useful, since
it allows us to *paritally* apply a function. We saw that in action with the add_n function.

In [88]:
multiply_by_3 = curried_multiply(3)
print(multiply_by_3(2))

6


An alternative to Currying is called *partial evaluation*.

In [89]:
def partial_eval(f, arg):
    def partial_f(x):
        return f(arg, x)
    return partial_f

multiply_by_4 = partial_eval(multiply, 4)
print(multiply_by_4(3))

12


In Python, we also have the builtin `functools` that provides us with a way to partially
apply any function, even if we haven't created it the way we did above.

In [90]:
from functools import partial

multiply_by_4 = partial(multiply, 4)
print(multiply_by_4(5))

20


I reccomend that you use the builtin `partial` function, since it offers more flexibility than our own `partial_eval` function.